# Helping Johann with eBay Car Sales Data

Johann Autoverkäufer is a serial entrepreneur that wants to get into the automotive world and has decided to open a second-hand car showroom in Germany. Johann hired us and said that he is still in his early stages, so he needs to understand the market first before building any inventory.

We don't know about his competitors, but we are aware that eBay Kleinanzeigen, the car used section of the German eBay website, is fairly popular, and extracting the data from it is easier than from other private selling competitors. 

After some research we are lucky to find a scraped dataset of 50,000 datapoints from https://data.world/data-society/used-cars-data. We will analyze the data using pandas and provide helpful insights for Johann.

## 1. Importing the database and transforming it into a dataframe

The data that we found is in a csv file. In order to analyze and provide the best insights out of it we need to convert it into a dataframe (table format) which will provide us with a structure that is much easier to analyze and in which the results will be much more visual.

We start by reading the csv file with pandas.

In [1]:
import pandas as pd

autos = pd.read_csv("autos.csv", encoding = "Latin-1")

# We used "Latin-1" encoding because the default "UTF-8" gave us an error
# meaning that it is not decoded as "UTF-8"

We continue by exploring what kind of information we have in our newly created "autos" dataframe

In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [3]:
first_lines_autos = autos.head()
print(first_lines_autos)

           dateCrawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offerType   price   abtest vehicleType  yearOfRegistration  \
0  privat   Angebot  $5,000  control         bus                2004   
1  privat   Angebot  $8,500  control   limousine                1997   
2  privat   Angebot  $8,990     test   limousine                2009   
3  privat   Angebot  $4,350  control  kleinwagen                2007   
4  privat   Angebot  $1,350     test       kombi                2003   

     gearbox  powerPS   model   odometer  monthOfRegistration fuelType  \
0    manuell      158  andere 

### 1.1 Initial Observations

#### We have 20 columns and 50.000 rows. At a first glance we can observe several things that might affect our analysis:

- There are several models, gearbox types and vehicle types missing. This could indicate, perhaps, that there are some vehicle parts also listed


- Missing values in fuel type could mean vehicles being sold without engine


- We also need to be careful with the missing values of notRepairedDamage. Does this mean that there is no damage, or that if there is damage is not repaired?


- One final interesting point is that the first 5 rows show 0 pictures. We might see patterns between vehicles with and without pictures


### 1.2 Format and readibility

We are going to be constantly making observations on the dataframe, so it is better if we make the data more readable. 

Right now, the notations in the column are camel case (written without spaces, with capital letter for each initial letter: "camelCase"). Code is generally more readable in snake case, with underscores as spaces and lower letters: "snake_case".

Let's rename the columns accordingly:

In [4]:
autos.rename({"dateCrawled" : "date_crawled"}, axis = 1, inplace = True)
autos.rename({"offerType" : "offer_type"}, axis = 1, inplace = True)
autos.rename({"vehicleType" : "vehicle_type"}, axis = 1, inplace = True)
autos.rename({"yearOfRegistration" : "registration_year"}, axis = 1, inplace = True)
autos.rename({"powerPS" : "power_ps"}, axis = 1, inplace = True)
autos.rename({"monthOfRegistration" : "registration_month"}, axis = 1, inplace = True)
autos.rename({"fuelType" : "fuel_type"}, axis = 1, inplace = True)
autos.rename({"notRepairedDamage" : "unrepaired_damage"}, axis = 1, inplace = True)
autos.rename({"dateCreated" : "ad_created"}, axis = 1, inplace = True)
autos.rename({"nrOfPictures" : "nr_pictures"}, axis = 1, inplace = True)
autos.rename({"postalCode" : "postal_code"}, axis = 1, inplace = True)
autos.rename({"lastSeen" : "last_seen"}, axis = 1, inplace = True)


Now if we examine the first five rows again we can see the result:

In [5]:
print(autos.head(10))

          date_crawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   
5  2016-03-21 13:47:45  Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...   
6  2016-03-20 17:55:21  VW_Golf_III_GT_Special_Electronic_Green_Metall...   
7  2016-03-16 18:55:19                               Golf_IV_1.9_TDI_90PS   
8  2016-03-22 16:51:34                                         Seat_Arosa   
9  2016-03-16 13:47:02          Renault_Megane_Scenic_1.6e_RT_Klimaanlage   

   seller offer_type   price   abtest vehicle_type  registration_year  \
0  privat    Angebot  $5,000  control          bus               2004   
1  pri

We changed the format to snake case, and this has improved readibility and brought much more consistency to the column names. Among the changes, we have also replaced some titles to sound easier to understand, such as:

- yearOfRegistration and monthOfRegistration by registration_year and registration_month
- notRepairedDamage by unrepaired_damage
- dateCreated by ad_created
- nrOfPictures by nr_pictures

Now that the format is more clear we can proceed with our analysis.

### 1.3 Exploring the data - Observations

We start our analysis by looking at our data more in depth. We will be using a very useful function in pandas that will also give us basic statistics on our database. This will help us assess if there is any data cleaning or formatting needed.

In [6]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-27 22:55:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


##### 1.3.1 Non Numeric Values

We see 'price' and 'odometer' have NaN values in the numeric statistics, when they should be numbers. If we look up the autos.info method above we also see that this is stored as an object, which means the values are strings. 

This could mean that price has currency symbols in its values, and the odometer kilometer values. 

Let's see:

In [7]:
print(autos["odometer"].value_counts())

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64


In [8]:
print(autos["price"].value_counts())

$0         1421
$500        781
$1,500      734
$2,500      643
$1,200      639
           ... 
$22,399       1
$129          1
$36,675       1
$25,300       1
$686          1
Name: price, Length: 2357, dtype: int64


Indeed, they have symbols in them. We will have to deal with them later.

##### 1.3.2 Odd values

If we now look at the "registration_year" column, we also see something very interesting: 

The minimum value is 1000 and its maximum is 9999. Unless there is a horse and a DeLorean time machine, this needs to be looked at.

In [9]:
print(autos["registration_year"].value_counts().sort_index())

1000    1
1001    1
1111    1
1500    1
1800    2
       ..
6200    1
8888    1
9000    2
9996    1
9999    4
Name: registration_year, Length: 97, dtype: int64


There is definitely something wrong with the data and we will need to do some cleanup later.

Now, if we look at the 'power_ps' column we see something similar: the maximum value is 17700 sp (horsepower). This doesn't look very correct. Let's see:

In [10]:
print(autos["power_ps"].value_counts())

0        5500
75       3171
60       2195
150      2046
140      1884
         ... 
650         1
490         1
362         1
153         1
16312       1
Name: power_ps, Length: 448, dtype: int64


In [11]:
print(autos.loc[autos["power_ps"] == 16312, :])

              date_crawled                  name  seller offer_type    price  \
35039  2016-03-10 08:57:25  Top_Cabrio_A4_s_line  privat    Angebot  $11,950   

      abtest vehicle_type  registration_year  gearbox  power_ps model  \
35039   test       cabrio               2008  manuell     16312    a4   

       odometer  registration_month fuel_type brand unrepaired_damage  \
35039  20,000km                   4    benzin  audi              nein   

                ad_created  nr_pictures  postal_code            last_seen  
35039  2016-03-10 00:00:00            0        51647  2016-03-17 09:16:19  


A quick Google search tells us that the Audi Top Cabrio A4 S Line has 163 hp approximately, so this is only an error in input.

## 2. Data Cleaning

### 2.1 "price" and "odometer" columns

#### 2.1.1 Transforming values to numbers

We start our data cleaning process with the columns for price and the odometer. Our first step is to remove any non-numeric values that we previously identified and then convert them to a number.

In [12]:
autos["price"] = autos["price"].str.replace("$", "")
autos["price"] = autos["price"].str.replace(",", "")
autos["price"] = autos["price"].astype(float)

In [13]:
print(autos["price"].head())

0    5000.0
1    8500.0
2    8990.0
3    4350.0
4    1350.0
Name: price, dtype: float64


In [14]:
autos["odometer"] = autos["odometer"].str.replace("km", "")
autos["odometer"] = autos["odometer"].str.replace(",", "")
autos["odometer"] = autos["odometer"].astype(float)

In [15]:
print(autos["odometer"].head())

0    150000.0
1    150000.0
2     70000.0
3     70000.0
4    150000.0
Name: odometer, dtype: float64


For both cases we have removed the letters and characters, leaving only the number, and afterwards converted this number to a float. In everyday life we always see decimal points in these two fields in any thinkable car selling site, hence our choice of float vs integer.

Since we have removed "km" from the odometer values and in Europe we also see odometers in miles, we are going to rename the column to reflect that the numbers are measured in km.

In [16]:
autos.rename({"odometer": "odometer_km"}, axis = 1, inplace = True)

A quick check on the first lines of the column confirms that the name has been changed correctly.

In [17]:
print(autos["odometer_km"].head(2))

0    150000.0
1    150000.0
Name: odometer_km, dtype: float64


#### 2.1.2 Finding and removing odd values

Now that both the "price" and "odometer_km" columns are numeric we are able to analyze the contents statistically and figure out if all the values are correct or if there is anything odd. We will start analysing by the column "price":

In [18]:
print(autos["price"].unique().shape)

(2357,)


We have way too many unique values, so we will need to look at other statistics to find out if there is anything odd:

In [19]:
print(autos["price"].describe())

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


We also see that the maximum value is very high, and way above the 75% percentile, so this is probably skewing our mean and will definitely skew future calculations. We proceed to verify what are the values on the top of this series:

In [20]:
print(autos["price"].value_counts().sort_index(ascending = False).head(20))

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
Name: price, dtype: int64


We see that the biggest jump in values starts after 350000 euros. On top of that there are less than 15 values, so if we removed them, in terms of amount of entries removed vs how much will the data be skewed this is a safe choice.

In [21]:
autos.loc[:,"price"] = autos[autos["price"].between(0, 350000)]

In [22]:
print(autos["price"].value_counts().sort_index(ascending = False).head(20))

350000.0    1
345000.0    1
299000.0    1
295000.0    1
265000.0    1
259000.0    1
250000.0    1
220000.0    1
198000.0    1
197000.0    1
194000.0    1
190000.0    1
180000.0    1
175000.0    1
169999.0    1
169000.0    1
163991.0    1
163500.0    1
155000.0    1
151990.0    1
Name: price, dtype: int64


In [23]:
print(autos["price"].describe())

count     49986.000000
mean       5721.525167
std        8983.617820
min           0.000000
25%        1100.000000
50%        2950.000000
75%        7200.000000
max      350000.000000
Name: price, dtype: float64


Now we can see that all the values are much more balanced and close to each other, which will give us more reliable results during the analysis. We do the same for the column "odometer_km"

In [24]:
print(autos["odometer_km"].unique().shape)

(13,)


This column is much easier to read, we can simply proceed to print them in a sorted shape:

In [25]:
print(autos["odometer_km"].value_counts().sort_index(ascending = False))

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
40000.0       819
30000.0       789
20000.0       784
10000.0       264
5000.0        967
Name: odometer_km, dtype: int64


All the values are values that you would expect in a used car site like this, so we do not need to remove any value.

### 2.2 Columns with dates and time

Before getting into cleaning the more obvious odd values that we observed earlier, we are going to study all the columns that contain dates:

- "date_crawled" - which indicates when was this ad found and indexed by eBay's software (crawlers) for the first time

- "ad_created" - when was the ad created

- "last seen" - when was this ad last seen by eBay's crawlers

- "registration_month" - month that the car was registered in

- "registration_year" - year that the car was registered in

We notice from our previous autos.info() function, that both the month and year of registration are stored in the dataframe as integer values. However, the other 3 dates are stored as strings.

Let's see how can we format the dates so that it is easier to analyze them:

#### 2.2.1 Dates as strings

#### 2.2.1.1 Data cleaning

In [26]:
print(autos[["date_crawled", "ad_created", "last_seen"]].head())

          date_crawled           ad_created            last_seen
0  2016-03-26 17:47:46  2016-03-26 00:00:00  2016-04-06 06:45:54
1  2016-04-04 13:38:56  2016-04-04 00:00:00  2016-04-06 14:45:08
2  2016-03-26 18:57:24  2016-03-26 00:00:00  2016-04-06 20:15:37
3  2016-03-12 16:58:10  2016-03-12 00:00:00  2016-03-15 03:16:28
4  2016-04-01 14:38:50  2016-04-01 00:00:00  2016-04-01 14:38:50


The dates each contain a very unique time of the day, which is great if we also wanted to set up an online site. However, Johann will set up shop physically so we don't need to be that specific about the hour and minute of the day, and he only needs the bigger picture. 

We notice that the first 10 characters of each column are the dates, so we can isolate those values and provide our first observations:

In [27]:
date_crawled_isolated = autos["date_crawled"].str[:10]
ad_created_isolated = autos["ad_created"].str[:10]
last_seen_isolated = autos["last_seen"].str[:10]

In [28]:
print(date_crawled_isolated.head())
print(ad_created_isolated.head())
print(last_seen_isolated.head())

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: date_crawled, dtype: object
0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: ad_created, dtype: object
0    2016-04-06
1    2016-04-06
2    2016-04-06
3    2016-03-15
4    2016-04-01
Name: last_seen, dtype: object


Now the visibility is much better and gives us an easier to read distribution. We can do a quick verification of the unique values for each column, and if they are not too many we can normalize data and draw some observations from it.

#### 2.2.1.2 Initial analysis and observations

In [29]:
date_crawled_isolated.unique().shape

(34,)

In [30]:
ad_created_isolated.unique().shape

(76,)

In [31]:
last_seen_isolated.unique().shape

(34,)

Strangely, we notice that there are more unique values for ad_created than for date_crawled and last_seen. We know that both of these two columns are imported by the crawlers, so it is possible that the difference in the count is for ads that have not been crawled yet, or that some of these ads were crawled at different dates from their creation. 

Let's see what we find if we analyze the normal distributions and sort the dates:

In [32]:
print(date_crawled_isolated.value_counts(normalize = True, dropna = False).sort_index())
print(ad_created_isolated.value_counts(normalize = True, dropna = False).sort_index())
print(last_seen_isolated.value_counts(normalize = True, dropna = False).sort_index())


2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64
2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
               ...   
2016-04-03    0.03892
2016-04-04    0.03688
2016-04-05    0.01184
2016-04-06    0.003

In [33]:
print(date_crawled_isolated.shape)
print(ad_created_isolated.shape)
print(last_seen_isolated.shape)

(50000,)
(50000,)
(50000,)


Interesting! We can already draw a couple of initial conclusions:

- All these columns have the same amount of values, yet the distribution tells us that some of these ads were created almost 6 months before they were crawled for the very first time. Could this mean that eBay did not introduce crawlers until 2016? This is a fun fact, but not specially relevant to Johann

- "last_seen" has the majority of its distribution in the latest dates of the database. If the crawlers go over ads by level of activity this could be an indicator of sales and an active second hand marketplace. This legitimizes the database and tells us that we are going to be giving useful information to Johann.

- "date_crawled" has a relatively big concentration during the March 20s, which in 2016 was around the date for Easter. This is an indicator that before and during a big holiday period is high season and it is also very relevant to Johann.

#### 2.2.2 Dates as integers

Now, if we check the distribution of our columns for "registration_year" and "registration_month", we see:

In [34]:
print(autos[["registration_month", "registration_year"]].describe())

       registration_month  registration_year
count        50000.000000       50000.000000
mean             5.723360        2005.073280
std              3.711984         105.712813
min              0.000000        1000.000000
25%              3.000000        1999.000000
50%              6.000000        2003.000000
75%              9.000000        2008.000000
max             12.000000        9999.000000


As we had initially observed, there is something wrong with the year as the minimum value is 1000 and the maximum is 9999. We also notice that the month also starts at number 0, which is wrong.

In [35]:
print(autos["registration_month"].value_counts().sort_index())

0     5075
1     3282
2     3008
3     5071
4     4102
5     4107
6     4368
7     3949
8     3191
9     3389
10    3651
11    3360
12    3447
Name: registration_month, dtype: int64


In [36]:
print(autos.loc[autos["registration_year"] > 2016, "registration_year"].value_counts().sort_index())

2017    1453
2018     492
2019       3
2800       1
4100       1
4500       1
4800       1
5000       4
5911       1
6200       1
8888       1
9000       2
9996       1
9999       4
Name: registration_year, dtype: int64


In [37]:
print(autos.loc[autos["registration_year"] > 2016, "registration_year"].shape)

(1966,)


We have issues in each column that could undermine the quality of our data:

- For "registration_month", 10% of our values are invalid, since they are 0

- For "registration_year", at least 4% of our values are also invalid, since it is impossible to list a car in 2016 that has been registered a year or more later. We also need to keep in mind those years that are too early.

We proceed to do as we did for the price. We will limit the years from 1900 (Ford T was invented 1908) to 2016, and the months from 1 to 12.

In [38]:
autos.loc[:,"registration_year"] = autos[autos["registration_year"].between(1900, 2016)]
autos.loc[:,"registration_month"] = autos[autos["registration_month"].between(1, 12)]


In [39]:
print(autos["registration_year"].describe())

count    48028.00000
mean      2002.80351
std          7.31085
min       1910.00000
25%       1999.00000
50%       2003.00000
75%       2008.00000
max       2016.00000
Name: registration_year, dtype: float64


In [40]:
print(autos["registration_month"].describe())

count    44925.000000
mean         6.369905
std          3.349160
min          1.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64


We now see that the values are very much in the range that you would expect, and that we still have a generous pool of 44.9 thousand values. Let's see what we find out if we look at the normal distribution:

In [41]:
print(autos["registration_year"].value_counts(normalize = True).sort_index())

1910.0    0.000187
1927.0    0.000021
1929.0    0.000021
1931.0    0.000021
1934.0    0.000042
            ...   
2012.0    0.027546
2013.0    0.016782
2014.0    0.013867
2015.0    0.008308
2016.0    0.027401
Name: registration_year, Length: 78, dtype: float64


In [42]:
print(autos["registration_month"].value_counts(normalize = True).sort_index())

1.0     0.073055
2.0     0.066956
3.0     0.112877
4.0     0.091308
5.0     0.091419
6.0     0.097229
7.0     0.087902
8.0     0.071029
9.0     0.075437
10.0    0.081269
11.0    0.074791
12.0    0.076728
Name: registration_month, dtype: float64


The normal distribution, linked with the previous statistics, tells us:

- Most of the cars are registered in the early 2000s (more than 50%) and there are few new cars
- March seems like a popular month but it does not tell us anything at this moment
- There are a few classics also sold online

## 3. Further analysis

### 3.1 Analysis by car brand - Pricing

Now that all our data is more clean, and we know that there is a decent amount of activity in the site, we can proceed to analyze the car by brand and find out how is each brand priced. We know that the mean pricing of each brand will be most likely be representing cars from the early 2000s as we pointed in the last section. This will be a very useful statistic for our client Johann to later apply to his business.

In [43]:
print(autos["brand"].value_counts(dropna=False))

volkswagen        10687
opel               5461
bmw                5429
mercedes_benz      4734
audi               4283
ford               3479
renault            2404
peugeot            1456
fiat               1308
seat                941
skoda               786
mazda               757
nissan              754
smart               701
citroen             701
toyota              617
sonstige_autos      546
hyundai             488
volvo               457
mini                424
mitsubishi          406
honda               399
kia                 356
alfa_romeo          329
porsche             294
suzuki              293
chevrolet           283
chrysler            181
dacia               129
daihatsu            128
jeep                110
subaru              109
land_rover           99
saab                 80
daewoo               79
trabant              78
jaguar               77
rover                69
lancia               57
lada                 31
Name: brand, dtype: int64


In [44]:
autos["brand"].describe()

count          50000
unique            40
top       volkswagen
freq           10687
Name: brand, dtype: object

The brands with 4 and 5 figure counts seem to represent a vast majority of the volume of 50.000 records, but we need to double check that this is a correct assumption.

In [45]:
#We use df[column].isin(['string']) to filter out the brands with most values
print(autos[autos["brand"].isin(["volkswagen", "opel", "bmw", "mercedes_benz", "audi", "ford", "renault", "peugeot", "fiat"])].shape)

(39241, 20)


34,5 thousand values is 70% of the values. Perhaps if we cover until the brands with 500 or less cars (down to 1% of the database each) we can get a more complete sample.

In [46]:
#We use df[column].isin(['string']) to filter out the brands with most values
print(autos[autos["brand"].isin(["volkswagen", "opel", "bmw", "mercedes_benz", "audi", "ford", "renault", "peugeot", "fiat", "seat", "skoda","mazda", "nissan", "citroen", "smart", "toyota"])].shape)

car_brands = autos[autos["brand"].isin(["volkswagen", "opel", "bmw", "mercedes-benz", "audi", "ford", "renault", "peugeot", "fiat", "seat", "skoda","mazda", "nissan", "citroen", "smart", "toyota"])]

(44498, 20)


45 thousand values is 80% of the database, enough for a representative sample. Now we will calculate the mean price per brand.

In [47]:
brand_prices_dictionary = {}

brands = ["volkswagen", "opel", "bmw", "mercedes_benz", "audi", "ford", "renault", "peugeot", "fiat", "seat", "skoda","mazda", "nissan", "citroen", "smart", "toyota"]

for b in brands:
    brand_match = autos[autos["brand"] == b]
    brand_match_price_sum = brand_match["price"].sum()
    brand_prices_dictionary[b] = round(brand_match_price_sum / brand_match.shape[0], 2)

print(brand_prices_dictionary)

    

{'volkswagen': 5157.34, 'opel': 2845.39, 'bmw': 8025.52, 'mercedes_benz': 8388.17, 'audi': 8965.56, 'ford': 3624.46, 'renault': 2351.3, 'peugeot': 3010.87, 'fiat': 2695.61, 'seat': 4219.43, 'skoda': 6305.04, 'mazda': 3962.54, 'nissan': 4588.88, 'citroen': 3681.4, 'smart': 3482.97, 'toyota': 5097.94}


We see that the prices start from 2.351 euros for Renault and go all the way to 8.000 euros for Audi. This is another very useful piece of information for Johann to position his shop.

We can also observe that there are 2 clear categories in the pricing:

- For mid-low range brands the price stays consistently between 2.500 euros and 3.500, except for Toyota
- All the high end brands start at least at 8.000 euros, which is a great insight should Johann target this segment
- We also have to remember that all these cars have an average age of 14 years based on the mean year of registration being early 2000s. This can help Johann learn more about the depreciation pace of different brands

### 3.2 Analysis by car brand - Mileage

Mileage is another strong factor when it comes to prices of used cars. It would also be very helpful for Johann to know what is the average mileage per brand in the eBay database, and then further compare price and mileage per brand.

In [48]:
brand_kilometers_dictionary = {}

brands = ["volkswagen", "opel", "bmw", "mercedes_benz", "audi", "ford", "renault", "peugeot", "fiat", "seat", "skoda","mazda", "nissan", "citroen", "smart", "toyota"]

for b in brands:
    brand_match = autos[autos["brand"] == b]
    brand_match_kilometer_sum = brand_match["odometer_km"].sum()
    brand_kilometers_dictionary[b] = round(brand_match_kilometer_sum / brand_match.shape[0], 2)

print(brand_kilometers_dictionary)

{'volkswagen': 128955.27, 'opel': 129298.66, 'bmw': 132521.64, 'mercedes_benz': 130886.14, 'audi': 129643.94, 'ford': 124131.93, 'renault': 128223.79, 'peugeot': 127352.34, 'fiat': 117037.46, 'seat': 122061.64, 'skoda': 110947.84, 'mazda': 125132.1, 'nissan': 118978.78, 'citroen': 119764.62, 'smart': 100756.06, 'toyota': 115988.65}


In order to compare these, we need to convert both the price and kilometer dictionares to a series, and later on put them inside a dataframe:

In [49]:
#We convert each dictionary into a series with the constructor pd.Series(dictionary)
price_series = pd.Series(brand_prices_dictionary)
kilometer_series = pd.Series(brand_kilometers_dictionary)

#We convert our series into a dataframe with the pd.Dataframe constructor. We start with price and add mean mileage
price_dataframe = pd.DataFrame(price_series, columns = ["mean_price"])
price_dataframe["mean_mileage"] = kilometer_series
print(price_dataframe)

               mean_price  mean_mileage
volkswagen        5157.34     128955.27
opel              2845.39     129298.66
bmw               8025.52     132521.64
mercedes_benz     8388.17     130886.14
audi              8965.56     129643.94
ford              3624.46     124131.93
renault           2351.30     128223.79
peugeot           3010.87     127352.34
fiat              2695.61     117037.46
seat              4219.43     122061.64
skoda             6305.04     110947.84
mazda             3962.54     125132.10
nissan            4588.88     118978.78
citroen           3681.40     119764.62
smart             3482.97     100756.06
toyota            5097.94     115988.65


From the resulting data we can see:

- Mileage is very similar in most of the cars

- That the high-end brands have the most mileage, yet they keep their price the highest. It shows that depreciation is not that strong for these and that brand plays a bigger part than mileage in pricing

### 3.3 Analysis by car brand - Model combinations

Finally, apart from price and mileage, another factor we would want to keep in mind is which brand / model combinations are listed. This will tell Johann what is the competition in the online marketplace and help him draft his strategy.

In [50]:
autos_by_brandmodel = autos.groupby(["brand", "model"])["brand"].count()
pd.set_option('display.max_rows', 291) #used to force Jupyter notebook to show all the values
print(autos_by_brandmodel)

brand          model             
alfa_romeo     145                      5
               147                     85
               156                     93
               159                     34
               andere                  61
               spider                  33
audi           100                     60
               200                      1
               80                     218
               90                       9
               a1                      84
               a2                      46
               a3                     882
               a4                    1291
               a5                     127
               a6                     835
               a8                      75
               andere                 220
               q3                      28
               q5                      62
               q7                      42
               tt                     149
bmw            1er                    538


In [51]:
autos_by_brandmodel_max = autos_by_brandmodel.sort_values()
print(autos_by_brandmodel_max)

brand          model             
ford           b_max                    1
rover          rangerover               1
bmw            i3                       1
rover          discovery                1
audi           200                      1
rover          freelander               2
lada           kalina                   2
lancia         kappa                    2
dacia          andere                   2
volvo          v60                      3
daihatsu       charade                  3
               materia                  4
land_rover     andere                   4
lada           samara                   4
fiat           croma                    5
land_rover     range_rover_evoque       5
alfa_romeo     145                      5
dacia          lodgy                    5
chrysler       crossfire                6
lancia         delta                    6
saab           9000                     6
volkswagen     amarok                   7
seat           exeo                     7


We show 2 different views:

1. The first one shows Johann what are the counts by brand and model categorizing everything by brand.
2. The second one shows the combinations of brands and models from less common to more common 

With this information, Johann can see clearly what is out there and how can he position his store.

# 4. Final Conclusions

To sum up, we want to give Johann a helicopter view of the findings of this report. These findings are:

1. This database is based in 2016. Johann might need to keep current inflation vs 2016's inflation in mind when taking points from this analysis


2. The last seen activity is very recent, which indicates that, at least in 2016, eBay was already a powerful driver of car sales. Johann will have to assume that there is still strong online competition


3. The vast majority of cars with odometer values are concentrated on mileages over 150.000 KM


4. There is a concentration of crawled ads on the March 20s dates, which suggests Easter is a high selling season


5. Most of the cars in the database have a registration year in the early 2000s


6. Mid-end brands stay consistently on the 2.500 to 3.500 range, and high-end brands stay consistently over 8.000 euros


7. High-end brands have the most mileage, however they keep the highest price. This is an indicator of slow devaluation


8. Range Rovers seem to be the rarest brand and model combinations, and the most common ones are, by far, Volkswagen Golf and BMW 3er


These are all key insights for Johann to have an starting point on his second-hand car business and we hope it helps me position himself more competitively. Wishing you all the best, Johann.